In [97]:
import os
import glob
import shutil

from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [98]:
os.getcwd()

'/home/nguyenminh/Documents/smart-speaker/mycroft-precise'

In [99]:
def build_data_tree(root_dir, data_name):
    data_path = os.path.join(root_dir, data_name)
    try:
        shutil.rmtree(data_path)
    except OSError as e:
        print("Error: %s : %s" % (data_path, e.strerror))
    os.makedirs(data_path)
    word = 'wake-word'
    not_word = 'not-wake-word'
    list_data_tree = [os.path.join(data_path, word),
                     os.path.join(data_path, 'test', word)]
    for d in list_data_tree:
        os.makedirs(d)
    return list_data_tree

In [101]:
ROOT_DIR = './data'
DATA_DIR = 'hey-sunshine'

In [102]:
train_word, test_word = build_data_tree(ROOT_DIR, DATA_DIR)

In [96]:
train_word, test_word

('./data/1-5-center/hey-sunshine/wake-word',
 './data/1-5-center/hey-sunshine/test/wake-word')

In [111]:
collect = [f for f in glob.glob('./data/hey-sunshine/macos' + "/**/*.wav", recursive=True)]

In [112]:
len(collect)

1083

In [113]:
train, test = train_test_split(collect, test_size=0.2)
len(train), len(test)

(866, 217)

In [114]:
# prepare wake-word
def migrate_data(root_list, new_path):
    for i, p in tqdm(enumerate(root_list)):
        file_name = p.split('/')[-1]
        shutil.copy(p, os.path.join(new_path, str(i) + '_' + file_name))
        
migrate_data(train, train_word)
migrate_data(test, test_word)

866it [00:00, 5098.63it/s]
217it [00:00, 6514.25it/s]


In [115]:
# prepare not-wake-word
src = './data/not-wake-word'
dest = './data/hey-sunshine'
train_not_wake_word = shutil.copytree(os.path.join(src, 'not-wake-word'), os.path.join(dest, 'not-wake-word'))
test_not_wake_word = shutil.copytree(os.path.join(src, 'test', 'not-wake-word'), os.path.join(dest, 'test', 'not-wake-word'))                   

## MFCC features

In [82]:
import numpy as np

from speechpy.feature import mfcc

from precise.util import load_audio

In [49]:
pr = {
    'window_t': 0.1, 'hop_t':0.05, 'buffer_t':1.5, 'sample_rate':16000,
    'sample_depth':2, 'n_mfcc':13, 'n_filt':20, 'n_fft':512, 'use_delta':False,
    'threshold_config':(6,4), 'threshold_center':0.2
}

In [50]:
AUDIO_PATH = './data/test-data'

In [67]:
list_test = [f for f in glob.glob(AUDIO_PATH + "**/*.wav")]
list_test

['./data/test-data/15db-minhcrop-heysunshinemacosmacos84_female_43_0932179617_43_normal_11.wav',
 './data/test-data/25db-minhcrop-heysunshinemacosmacos84_female_43_0932179617_43_normal_11.wav',
 './data/test-data/minhcrop-heysunshinemacosmacos84_female_43_0932179617_43_normal_11.wav',
 './data/test-data/30db-minhcrop-heysunshinemacosmacos84_female_43_0932179617_43_normal_11.wav']

In [68]:
x = []
for p in list_test:
    x.append(mfcc(load_audio(p), pr["sample_rate"], pr['window_t'], pr['hop_t'], pr['n_mfcc'], pr['n_filt'], pr['n_fft']))

In [58]:
mfcc(x, pr["sample_rate"], pr['window_t'], pr['hop_t'], pr['n_mfcc'], pr['n_filt'], pr['n_fft'])

array([[-3.60436534e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e

In [72]:
len(x[0])

28

In [89]:
sub = x[2] - x[3]
sub.shape, x[2].shape

((28, 13), (28, 13))

In [91]:
np.abs(np.true_divide(sub, x[2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[0.00000000e+00,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan],
       [0.00000000e+00,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan],
       [0.00000000e+00,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan],
       [0.00000000e+00,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
                   nan],
       [0.00000000e+00,            n

In [94]:
l = [1,2,3,4,5].remove(l[-1])
l

TypeError: 'int' object is not subscriptable